The Displaybot should show a window on a small wall-mounted display that plays gifs and videos from a telegram group or tunes to a web radio station.

First, I need to create a Telegram bot. For this, install the Python Telegram Bot library and the peewee database ORM with

    $ pip install peewee python-telegram-bot sqlite3 ffmpy --upgrade

and then setup logging. I follow the [echobot example](https://github.com/python-telegram-bot/python-telegram-bot/blob/master/examples/echobot2.py).

Also, setup your Telegram api token below. Get a token by talking to [this bot](https://telegram.me/botfather) on Telegram.

In [17]:
import logging
import requests
import json
import tempfile
import ffmpy
import datetime
import peewee

from requests.exceptions import RequestException
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
from playhouse.sqlite_ext import SqliteExtDatabase

logging.basicConfig(level=logging.DEBUG,
                    format='%(levelname)s - %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

TELEGRAM_API_TOKEN = "YOUR TOKEN HERE"

# This will be the database of video clips
DATABASE_FILENAME = "memory.db"

# This cache file is used to feed the webserver with data
CACHE_LOCATION = "frontend/public/data.json"

# As anyone will be able to add the bot and add pictures to your display,
# you can filter telegram usernames here
ALLOWED_USERS = []

SERVER_URL = "http://localhost:3000"

Define a few command handlers for Telegram. These usually take the two arguments bot and
update. Error handlers also receive the raised TelegramError object in error.

The start command is sent when the bot is started.

In [18]:
def start(bot, update):
    update.message.reply_text('Gimme dat gif. Send an .mp4 link!')

Handle errors, just in case

In [19]:
def error(bot, update, error):
    logger.warn('Update "%s" caused error "%s"' % (update, error))

Next ist the receiver for our app. It will look at incoming messages and determine, whether they contain a link and then wether that link points at an mp4 video. This will then be added to the database for display.

There are special cases:
- if `url` ends in `gifv`, that is rewritten to `mp4`
- if `url` ends in `gif`, the gif is downloaded and converted to a local `mp4` (see code for that below)

In [20]:
def receive(bot, update):
    elems = update.message.parse_entities(types=["url"])
    logger.info("Incoming message with {} entities".format(len(elems)))

    for elem in elems:
        url = update.message.text[elem.offset:(elem.offset + elem.length)]

        # Rewrite gifv links extension and try that
        if url[-4:] == "gifv":
            url = url[:-4] + "mp4"
            logger.info("Rewrite .gifv to {}".format(url))

        # Convert gif files using ffmpeg
        if url[-3:] == "gif":
            url = convert_gif(url)

        try:
            link = requests.head(url)

        except RequestException:
            logger.warning("Link not valid")
            update.message.reply_text("Link not valid")

        else:
            if "Content-Type" in link.headers and link.headers["Content-Type"] in ["video/mp4", "video/webm"]:
                if add_url(url=url, author=update.message.from_user.first_name):
                    update.message.reply_text("Added video to database")
                else:
                    update.message.reply_text("Reposter!")

            else:
                logger.info("Link not supported: {}".format(link.headers))

In order to convert gifs to the less ressource intensive mp4 format, we can use the ffmpy library, which calls ffmpeg for us outside of python, to make the conversion. 

This function creates a temporary file and writes the gif to it. Then ffmpeg is called with settings for converting a gif to an mp4 and the result is stored in `frontend/public/videos/`, where the frontend script will be able to access it. 

In [21]:
def convert_gif(url):
    rv = False
    temp = tempfile.NamedTemporaryFile()
    r = requests.get(url, stream=True)
    if r.ok:
        logger.info("Downloading gif to {}...".format(temp.name))
        for block in r.iter_content(1024):
            temp.write(block)

        logger.info("Converting...")
        fname = url.split("/")[-1]
        ff = ffmpy.FFmpeg(
            inputs={temp.name: None},
            outputs={'frontend/public/videos/{}.mp4'.format(fname): '-movflags faststart -pix_fmt yuv420p -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2"'}
        )
        ff.run()
        rv = "{}/videos/{}.mp4".format(SERVER_URL, fname)
        logger.info(rv)
    return rv

Now setup a local database, handled by the Peewee ORM, which allows us to simply handle Python objects for db access instead of writing SQL queries.

In [22]:
db = SqliteExtDatabase(DATABASE_FILENAME)

class BaseModel(peewee.Model):
    class Meta:
        database = db

class Video(BaseModel):
    url = peewee.CharField(unique=True)
    created = peewee.DateTimeField(default=datetime.datetime.now)
    author = peewee.CharField()

    def __repr__(self):
        return "<Video by '{}' at '{}' />".format(self.author, self.url)

    def serialize(self):
        rv = {
            "url": self.url,
            "author": self.author,
            "created": self.created.isoformat()
        }
        return rv


# Connect to the database and create tables for the models
db.connect()
try:
    db.create_tables([Video])
except peewee.OperationalError:
    logger.info("Tables already exist")

INFO:root:Tables already exist


Then write a handler to store received videos in the database and computes a cached JSON response on disk with all current videos 

In [24]:
def add_url(url, author):
    try:
        video = Video.create(url=url, author=author)
    except IntegrityError:
        logger.info("Video already exists {}".format(url))
        video = None
    else:
        logger.info("Stored new Video {}".format(video))
        refresh_cache()
        return video

def refresh_cache():
    videos = Video.select().order_by(Video.created.desc())
    rv = {
        "videos": {v.id: v.serialize() for v in videos},
        "config": None
    }
    with open(CACHE_LOCATION, "w") as f:
        json.dump(rv, f)
    logger.info("Cache refreshed. Total {} videos".format(len(rv["videos"])))

Add the main  function, where the handler functions above are registered with the Telegram Bot API and continous polling for new messages as well as the flask server are started.

In [25]:
def main():
    # Create the EventHandler and pass it your bot's token.
    updater = Updater(TELEGRAM_API_TOKEN)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # on different commands - answer in Telegram
    dp.add_handler(CommandHandler("start", start))

    # on noncommand i.e message - echo the message on Telegram
    dp.add_handler(MessageHandler(None, receive))

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    # Run the bot until the you presses Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

Start your bot by saving this notebook as `display-bot.py` and running `$ python display-bot.py`.

Before the main loop is started, database contents are dumped to the cache file, accessible by the frontend script.

In [ ]:
if __name__ == '__main__':
    refresh_cache()
    main()

INFO:root:Cache refreshed. Total 20 videos
ERROR:telegram.ext.updater:Error while getting Updates: Conflict: terminated by other long poll or webhook (409)
ERROR:telegram.ext.updater:Error while getting Updates: Conflict: terminated by other long poll or webhook (409)
ERROR:telegram.ext.updater:Error while getting Updates: Conflict: terminated by other long poll or webhook (409)
ERROR:telegram.ext.updater:Error while getting Updates: Conflict: terminated by other long poll or webhook (409)
ERROR:telegram.ext.updater:Error while getting Updates: Conflict: terminated by other long poll or webhook (409)
ERROR:telegram.ext.updater:Error while getting Updates: Conflict: terminated by other long poll or webhook (409)
ERROR:telegram.ext.updater:Error while getting Updates: Conflict: terminated by other long poll or webhook (409)


In [ ]:
main()